In [31]:
## imports
import pandas as pd
import time
import numpy as np

In [32]:
# Define the function to load data
def load_team_data(base_url, seasons):
    team_df = pd.DataFrame()
    
    for season in seasons:
        url = base_url.format(season)
        try:
            # Read the data from the URL
            season_df = pd.read_html(url, attrs={"id": "matchlogs_for"})[0]
            
            # Add a column for the season
            season_df['Season'] = season
            
            # Concatenate the season's data to the main DataFrame
            team_df = pd.concat([team_df, season_df], ignore_index=True)
            
            print(f"Successfully loaded data for season {season}")
        except Exception as e:
            print(f"Failed to load data for season {season}: {e}")
        
        # Sleep for 5 seconds to avoid triggering a "too many requests" error
        time.sleep(5)
    
    return team_df

In [33]:
# Define the seasons
seasons = [f"{year}-{year+1}" for year in range(1996, 2024)]

# Base URL format for the Champions League stats
real_madrid_base_url = 'https://fbref.com/en/squads/53a2f082/{}/c8/Real-Madrid-Stats-Champions-League'
dortmund_base_url  = 'https://fbref.com/en/squads/add600ae/{}/c8/Dortmund-Stats-Champions-League'

# Initialize an empty DataFrame to hold all the data
realMadrid_df = pd.DataFrame()
dortmund_df = pd.DataFrame()

# Load data for Real Madrid
realMadrid_df = load_team_data(real_madrid_base_url, seasons)

# Load data for Dortmund
dortmund_df = load_team_data(dortmund_base_url, seasons)

# Display a sample of the loaded data
print(realMadrid_df.head())
print(dortmund_df.head())

# Save the combined DataFrames to CSV files for future use
realMadrid_df.to_csv('real_madrid_champions_league_1996_2024.csv', index=False)
dortmund_df.to_csv('dortmund_champions_league_1996_2024.csv', index=False)

Successfully loaded data for season 1996-1997
Successfully loaded data for season 1997-1998
Successfully loaded data for season 1998-1999
Successfully loaded data for season 1999-2000
Successfully loaded data for season 2000-2001
Successfully loaded data for season 2001-2002
Successfully loaded data for season 2002-2003
Successfully loaded data for season 2003-2004
Successfully loaded data for season 2004-2005
Successfully loaded data for season 2005-2006
Successfully loaded data for season 2006-2007
Successfully loaded data for season 2007-2008
Successfully loaded data for season 2008-2009
Successfully loaded data for season 2009-2010
Successfully loaded data for season 2010-2011
Successfully loaded data for season 2011-2012
Successfully loaded data for season 2012-2013
Successfully loaded data for season 2013-2014
Successfully loaded data for season 2014-2015
Successfully loaded data for season 2015-2016
Successfully loaded data for season 2016-2017
Successfully loaded data for seaso

In [38]:
def preprocess_team_data(df, output_filename):
    # Drop rows where 'Comp' is not NaN (assuming you want only Champions League data)
    df = df[df['Comp'].isna()]
    
    # Drop rows where 'Result' is NaN
    df = df.dropna(subset=['Result'])
    
    # Drop the 'Notes' column
    if 'Notes' in df.columns:
        df = df.drop(columns=['Notes'])
    
    # Display DataFrame information
    print(df.info())
    
    # Save the preprocessed DataFrame to a CSV file
    df.to_csv(output_filename, index=False)
    
    return df

In [39]:
realMadrid_df = pd.read_csv('real_madrid_champions_league_1996_2024.csv')
realMadrid_df = preprocess_team_data(realMadrid_df, 'real_madrid_champions_league_1996_2024.csv')

dortmund_df = pd.read_csv('dortmund_champions_league_1996_2024.csv')
dortmund_df = preprocess_team_data(dortmund_df, 'dortmund_champions_league_1996_2024.csv')

<class 'pandas.core.frame.DataFrame'>
Index: 310 entries, 42 to 421
Data columns (total 19 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date          310 non-null    object 
 1   Time          116 non-null    object 
 2   Comp          0 non-null      object 
 3   Round         310 non-null    object 
 4   Day           310 non-null    object 
 5   Venue         310 non-null    object 
 6   Result        310 non-null    object 
 7   GF            310 non-null    object 
 8   GA            310 non-null    object 
 9   Opponent      310 non-null    object 
 10  Poss          116 non-null    float64
 11  Attendance    102 non-null    float64
 12  Captain       104 non-null    object 
 13  Formation     129 non-null    object 
 14  Referee       310 non-null    object 
 15  Match Report  310 non-null    object 
 16  Season        310 non-null    object 
 17  xG            78 non-null     float64
 18  xGA           78 non-null     floa

## **Processing the Data**

In [44]:
import pandas as pd

# Define the function to preprocess team data
def preprocess_team_goals(df):
    # Function to split goals into regular and penalty shootout scores
    def split_goals(goal):
        try:
            if '(' in goal:
                regular, penalty = goal.split('(')
                penalty = penalty.rstrip(')')
                return int(float(regular)), int(float(penalty))
            else:
                return int(float(goal)), 0
        except ValueError:
            return 0, 0  # Handle cases where conversion fails

    # Apply the function to split GF and GA
    df[['GF', 'GF_pen']] = df['GF'].astype(str).apply(split_goals).apply(pd.Series)
    df[['GA', 'GA_pen']] = df['GA'].astype(str).apply(split_goals).apply(pd.Series)

    # Rename the columns for clarity
    df.rename(columns={'GF': 'GF_reg', 'GA': 'GA_reg'}, inplace=True)

    # Display the updated DataFrame
    print(df[['GF_reg', 'GF_pen', 'GA_reg', 'GA_pen']].head())

    return df



In [45]:
# Example usage for Real Madrid data
realMadrid_df = pd.read_csv('real_madrid_champions_league_1996_2024.csv')
realMadrid_df = preprocess_team_goals(realMadrid_df)

# Example usage for Dortmund data
dortmund_df = pd.read_csv('dortmund_champions_league_1996_2024.csv')
dortmund_df = preprocess_team_goals(dortmund_df)

   GF_reg  GF_pen  GA_reg  GA_pen
0       4       0       1       0
1       2       0       0       0
2       5       0       1       0
3       0       0       0       0
4       0       0       2       0
   GF_reg  GF_pen  GA_reg  GA_pen
0       2       0       1       0
1       3       0       0       0
2       1       0       0       0
3       1       0       2       0
4       2       0       2       0


#### **Some Variables Explanation**
- Home Team Form : 
  - Definition: Home team form represents the average performance of the team over the last few matches.
  - Purpose: To capture the team's recent performance, which is often a good indicator of how well the team is likely to perform in the next match.
  - Why 5 Games?:
      - Balance: A 5-game window strikes a balance between being too short (which might make the average overly sensitive to short-term fluctuations) and too long (which might dilute the impact of recent form).
      - Common Practice: It's a common practice in sports analytics to use a 5-game window to represent recent form. This is because 5 games provide a reasonable sample size to assess the current state of the team while still reflecting recent changes in performance.
      - Practical: It covers roughly one month of matches for teams that play weekly, which is a common in futbol
  
- Goal Difference Rolling:
  - Definition: Goal difference rolling is the average goal difference over the last few matches.
  - Purpose: To capture the recent scoring and defensive performance of the team, which can be an indicator of the team's strength and balance between offense and defense.

In [47]:
# Create match outcome: 1 for win, 0 for lose/draw based on the final result including penalties
def determine_match_outcome(row):
    if row['GF_reg'] > row['GA_reg']:
        return 1
    elif row['GF_reg'] < row['GA_reg']:
        return 0
    else:  # If regular time is a draw, determine based on penalties
        if row['GF_pen'] > row['GA_pen']:
            return 1
        else:
            return 0

def preprocess_team_data(df,output_filename):

    # Create match outcome: 1 for win, 0 for lose/draw based on the final result including penalties
    df['match_outcome'] = df.apply(determine_match_outcome, axis=1)

    # Create goal difference using regular time goals
    df['goal_difference'] = df['GF_reg'] - df['GA_reg']

    # Create a binary feature for home advantage
    df['home_advantage'] = df['Venue'].apply(lambda x: 1 if x == 'Home' else 0)

    # Calculate rolling average of recent match outcomes (form)
    df = df.sort_values(by='Date')
    df['home_team_form'] = df['match_outcome'].rolling(window=5).mean().shift(1).fillna(0)

    # Calculate rolling average of goal difference
    df['goal_difference_rolling'] = df['goal_difference'].rolling(window=5).mean().shift(1).fillna(0)

    # Fill missing values for numeric columns with 0
    numeric_cols = ['Poss', 'Attendance', 'xG', 'xGA', 'GF_reg', 'GA_reg', 'GF_pen', 'GA_pen']
    df[numeric_cols] = df[numeric_cols].fillna(0)

    # For categorical columns, you can fill with a placeholder or the most frequent value
    categorical_cols = ['Time', 'Captain', 'Formation']
    df[categorical_cols] = df[categorical_cols].fillna('Unknown')

     # Save the preprocessed DataFrame to a CSV file
    df.to_csv(output_filename, index=False)

    return df      


In [48]:
realMadrid_df = preprocess_team_data(realMadrid_df, 'real_madrid_champions_league_1996_2024.csv')
dortmund_df = preprocess_team_data(dortmund_df, 'dortmund_champions_league_1996_2024.csv')

# Display a sample of the preprocessed data
print(realMadrid_df[['goal_difference', 'home_team_form', 'goal_difference_rolling', 'match_outcome']].head())
print(dortmund_df[['goal_difference', 'home_team_form', 'goal_difference_rolling', 'match_outcome']].head())

   goal_difference  home_team_form  goal_difference_rolling  match_outcome
0                3             0.0                      0.0              1
1                2             0.0                      0.0              1
2                4             0.0                      0.0              1
3                0             0.0                      0.0              0
4               -2             0.0                      0.0              0
   goal_difference  home_team_form  goal_difference_rolling  match_outcome
0                1             0.0                      0.0              1
1                3             0.0                      0.0              1
2                1             0.0                      0.0              1
3               -1             0.0                      0.0              0
4                0             0.0                      0.0              0


## **Generating Synthetic Data**
Using Monte Carlo simulations to generate synthetic match data based on the derived features.

In [ ]:
import numpy as np
import pandas as pd

# Function to generate synthetic match data using Monte Carlo simulation
def generate_synthetic_matches_monte_carlo(n, avg_gf, avg_ga, avg_poss, avg_form, avg_goal_diff):
    synthetic_data = []
    for _ in range(n):
        home_goals = np.random.poisson(avg_gf)
        away_goals = np.random.poisson(avg_ga)
        match_outcome = 1 if home_goals > away_goals else 0
        goal_difference = home_goals - away_goals
        poss = np.random.normal(avg_poss, 5)  # Assuming normal distribution with some variation
        form = np.random.normal(avg_form, 0.1)  # Assuming normal distribution with some variation
        goal_diff_rolling = np.random.normal(avg_goal_diff, 1)  # Assuming normal distribution with some variation
        home_advantage = np.random.choice([0, 1])  # Randomly assign home or away
        competition_importance = np.random.choice([0, 1], p=[0.8, 0.2])  # Assume most matches are not high importance

        synthetic_data.append([home_goals, away_goals, goal_difference, match_outcome, poss, form, goal_diff_rolling, home_advantage, competition_importance])
    
    return pd.DataFrame(synthetic_data, columns=['home_goals', 'away_goals', 'goal_difference', 'match_outcome', 'Poss', 'home_team_form', 'goal_difference_rolling', 'home_advantage', 'competition_importance'])

# Calculate averages for generating synthetic data
avg_gf = realMadrid_df['GF_reg'].mean()
avg_ga = realMadrid_df['GA_reg'].mean()
avg_poss = realMadrid_df['Poss'].mean()
avg_form = realMadrid_df['home_team_form'].mean()
avg_goal_diff = realMadrid_df['goal_difference_rolling'].mean()

# Generate synthetic matches
synthetic_matches = generate_synthetic_matches_monte_carlo(100, avg_gf, avg_ga, avg_poss, avg_form, avg_goal_diff)
print(synthetic_matches.head())
